In [1]:
#%matplotlib widget
%matplotlib ipympl

In [2]:
# initialization
from rayoptics.environment import *
from rayoptics.optical.elements import create_mirror
from rayoptics.optical.model_enums import DecenterType as dec

In [3]:
root_pth = Path(rayoptics.__file__).resolve().parent

In [4]:
app = AppManager(None)

In [5]:
#app.model = open_model(root_pth/"codev/tests/ag_dblgauss.seq")
app.model = OpticalModel()
opm = app.model
sm = opm.seq_model
osp = opm.optical_spec

### Define first order aperture and field for model

In [6]:
osp.pupil = PupilSpec(osp, pupil_type=PupilType.EPD, value=1.)
osp.field_of_view = FieldSpec(osp, field_type=FieldType.OBJ_ANG, flds=[0.])
osp.spectral_region = WvlSpec([(555.0, 1.0)], ref_wl=0)

### Enter sequential model

In [7]:
opm.radius_mode = True

sm.gaps[0].thi=1e10

sm.add_surface([0., 5.0])
sm.set_stop()

seq_m1, me1 = create_mirror()
m1 = seq_m1[0][0]
sm += m1
sm += Gap(t=-5.0)
opm.ele_model.add_element(me1[0])
m1.decenter = srf.DecenterData(dec.BEND, alpha=60.)

seq_m2, me2 = create_mirror()
m2 = seq_m2[0][0]
sm += m2
sm += Gap(t=5.0)
me2[0].z_dir = -1.0
opm.ele_model.add_element(me2[0])
m2.decenter = srf.DecenterData(dec.BEND, alpha=-60.)

seq_m3, me3 = create_mirror()
m3 = seq_m3[0][0]
sm += m3
sm += Gap(t=-5.0)
opm.ele_model.add_element(me3[0])
m3.decenter = srf.DecenterData(dec.BEND, alpha=-45.)


In [8]:
me3[0]

In [9]:
opm.update_model()

In [10]:
opm.ele_model.list_elements()

0: Mirror (Mirror): Mirror: Spherical(c=0.0), t=0.0500, sd=1.0000
1: Mirror (Mirror): Mirror: Spherical(c=0.0), t=0.0500, sd=1.0000
2: Mirror (Mirror): Mirror: Spherical(c=0.0), t=0.0354, sd=0.7071


In [11]:
for i,trfm in enumerate(sm.gbl_tfrms):
    print("{}: {:12.7g} {:12.7g} {:12.7g}".format(i, trfm[1][0], trfm[1][1], trfm[1][2]))

0:            0            0       -1e+10
1:            0            0            0
2:            0            0            5
3:            0    -4.330127          7.5
4:            0    -4.330127         12.5
5:            0     0.669873         12.5


In [12]:
pt0 = np.array([0., 0., 0.])
dir0 = np.array([0., 0., 1.])
wvl = sm.central_wavelength()
oar = rt.trace(sm, pt0, dir0, wvl)
list_ray(oar[0])

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000            0     0.000000     0.000000     1.000000        1e+10
  1:      0.00000      0.00000            0     0.000000     0.000000     1.000000            5
  2:      0.00000      0.00000            0     0.000000    -0.866025    -0.500000            5
  3:      0.00000      0.00000            0     0.000000    -0.866025     0.500000            5
  4:      0.00000      0.00000            0     0.000000     0.707107    -0.707107            5
  5:      0.00000      0.00000            0     0.000000     0.000000    -1.000000            0


In [13]:
tfrms = sm.gbl_tfrms
print("            X            Y            Z          SRL"
      "           SRM          SRN")
for i, r in enumerate(oar[mc.ray][0:]):
    rot, trns = tfrms[i]
    p = rot.dot(r[mc.p]) + trns
    snr = rot.dot(r[mc.nrml])
    print("{:3d}: {:12.5f} {:12.5f} {:12.5g} {:4.0f} {:12.6f} {:12.6f} {:12.6f}"
          .format(i, p[0], p[1], p[2], sm.z_dir[i], snr[0], snr[1], snr[2]))

            X            Y            Z          SRL           SRM          SRN
  0:      0.00000      0.00000       -1e+10    1     0.000000     0.000000     1.000000
  1:      0.00000      0.00000            0    1     0.000000     0.000000     1.000000
  2:      0.00000      0.00000            5   -1     0.000000     0.866025     0.500000
  3:      0.00000     -4.33013          7.5    1     0.000000     0.866025     0.500000
  4:      0.00000     -4.33013         12.5   -1     0.000000    -0.707107     0.707107
  5:      0.00000      0.66987         12.5   -1     0.000000    -1.000000     0.000000


In [14]:
list_ray(oar,tfrms=sm.gbl_tfrms)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000       -1e+10     0.000000     0.000000     1.000000        1e+10
  1:      0.00000      0.00000            0     0.000000     0.000000     1.000000            5
  2:      0.00000      0.00000            5     0.000000    -0.866025     0.500000            5
  3:      0.00000     -4.33013          7.5     0.000000     0.000000     1.000000            5
  4:      0.00000     -4.33013         12.5     0.000000     1.000000     0.000000            5
  5:      0.00000      0.66987         12.5     0.000000     1.000000    -0.000000            0


In [15]:
sm.z_dir

[1.0, 1.0, -1.0, 1.0, -1.0, -1.0]

In [16]:
layout_plt = plt.figure(FigureClass=LensLayoutFigure, opt_model=opm).plot()
#app.add_view(layout_plt, ModelInfo(opm, layout_plt.refresh))

FigureCanvasNbAgg()

In [17]:
sm.list_decenters()

0 Gap(t=10000000000.0, medium=Air)
1 Gap(t=5.0, medium=Air)
2 Gap(t=-5.0, medium=Air)
  'BEND': Decenter: array([0., 0., 0.]), Tilt: array([60.,  0.,  0.])
3 Gap(t=5.0, medium=Air)
  'BEND': Decenter: array([0., 0., 0.]), Tilt: array([-60.,   0.,   0.])
4 Gap(t=-5.0, medium=Air)
  'BEND': Decenter: array([0., 0., 0.]), Tilt: array([-45.,   0.,   0.])


In [18]:
opm.save_model('Tilted mirrors')

In [19]:
opm = open_model(root_pth/"codev/tests/folded_lenses.seq")
sm = opm.seq_model
osp = opm.optical_spec
pm = opm.parax_model

In [20]:
layout_plt = plt.figure(FigureClass=LensLayoutFigure, opt_model=opm).plot()

FigureCanvasNbAgg()

In [21]:
for i,s in enumerate(sm.ifcs):
    print(i, sm.z_dir[i], s.profile_cv)

0 1.0 0.0
1 1.0 0.0
2 -1.0 0.0
3 -1.0 -0.67
4 -1.0 0.0
5 -1.0 0.0
6 -1.0 -0.77095
7 1.0 0.0
8 1.0 -0.67
9 1.0 0.0
10 1.0 0.0
11 1.0 -0.59225
12 -1.0 0.0
13 -1.0 0.0


In [22]:
em = opm.ele_model
em.list_elements()

0: Aperture Stop (DummyInterface): Surface(profile=Spherical(c=0.0))
1: AirGap Aperture Stop-M1 (AirGap): Gap(t=5.0, medium=Air)
2: M1 (Mirror): Mirror: Spherical(c=0.0), t=0.0500, sd=1.0000
3: AirGap M1-E1 (AirGap): Gap(t=-2.25, medium=Air)
4: E1 (Element): Element: Spherical(c=-0.67), Spherical(c=0.0), t=-0.2500, sd=0.5000, glass: N-BK7
5: AirGap E1-E2 (AirGap): Gap(t=-0.05, medium=Air)
6: E2 (Element): Element: Spherical(c=0.0), Spherical(c=-0.77095), t=-0.2500, sd=0.4712, glass: N-BK7
7: AirGap E2-M3 (AirGap): Gap(t=-2.25, medium=Air)
8: M3 (Mirror): Mirror: Spherical(c=0.0), t=0.0431, sd=0.8620
9: AirGap M3-E3 (AirGap): Gap(t=2.25, medium=Air)
10: E3 (Element): Element: Spherical(c=-0.67), Spherical(c=0.0), t=0.2500, sd=0.4596, glass: N-BK7
11: AirGap E3-E4 (AirGap): Gap(t=0.05, medium=Air)
12: E4 (Element): Element: Spherical(c=0.0), Spherical(c=-0.59225), t=0.2500, sd=0.4852, glass: N-BK7
13: AirGap E4-M5 (AirGap): Gap(t=2.25, medium=Air)
14: M5 (Mirror): Mirror: Spherical(c=0.0

In [23]:
m1 = em.elements[0]

In [24]:
str(m1)

'Surface(profile=Spherical(c=0.0))'

In [25]:
m1.tfrm

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]), array([0., 0., 0.]))

In [26]:
m2 = em.elements[3]

In [27]:
str(m2)

'Gap(t=-2.25, medium=Air)'

In [28]:
m2.tfrm

(array([[ 1.       ,  0.       ,  0.       ],
        [ 0.       ,  0.5      ,  0.8660254],
        [ 0.       , -0.8660254,  0.5      ]]), array([0., 0., 5.]))

In [29]:
for i,e in enumerate(em.elements):
    rifc_indx = sm.ifcs.index(e.reference_interface())
    print(i, rifc_indx, e.tfrm[0].T[2], e.reference_interface().profile.normal(np.array([0., 0., 0.])))

0 1 [0. 0. 1.] [-0. -0.  1.]
1 1 [0. 0. 1.] [-0. -0.  1.]
2 2 [0.        0.8660254 0.5      ] [-0. -0.  1.]
3 2 [0.        0.8660254 0.5      ] [-0. -0.  1.]
4 3 [ 0.         0.8660254 -0.5      ] [0. 0. 1.]
5 4 [ 0.         0.8660254 -0.5      ] [-0. -0.  1.]
6 5 [ 0.         0.8660254 -0.5      ] [-0. -0.  1.]
7 6 [ 0.         0.8660254 -0.5      ] [0. 0. 1.]
8 7 [0.        0.8660254 0.5      ] [-0. -0.  1.]
9 7 [0.        0.8660254 0.5      ] [-0. -0.  1.]
10 8 [ 0.00000000e+00 -1.48741681e-17  1.00000000e+00] [0. 0. 1.]
11 9 [ 0.00000000e+00 -1.48741681e-17  1.00000000e+00] [-0. -0.  1.]
12 10 [ 0.00000000e+00 -1.48741681e-17  1.00000000e+00] [-0. -0.  1.]
13 11 [ 0.00000000e+00 -1.48741681e-17  1.00000000e+00] [0. 0. 1.]
14 12 [ 0.         -0.70710678  0.70710678] [-0. -0.  1.]
15 12 [ 0.         -0.70710678  0.70710678] [-0. -0.  1.]
16 13 [ 0.00000000e+00 -1.00000000e+00 -1.01465364e-17] [-0. -0.  1.]


In [30]:
m2.tfrm[0][2]

array([ 0.       , -0.8660254,  0.5      ])

In [31]:
d = m2.tfrm[0].dot(np.array([0., 0., 1.])); d

array([0.       , 0.8660254, 0.5      ])

In [32]:
m2.tfrm[0].T[2]

array([0.       , 0.8660254, 0.5      ])

In [33]:
m2.tfrm[0].transpose()[2]

array([0.       , 0.8660254, 0.5      ])

In [34]:
m2_indx = sm.ifcs.index(m2.reference_interface()); m2_indx

2

In [35]:
sm.gbl_tfrms[m2_indx][0].T

array([[ 1.       ,  0.       ,  0.       ],
       [ 0.       ,  0.5      , -0.8660254],
       [ 0.       ,  0.8660254,  0.5      ]])

In [36]:
m2.reference_interface().profile.normal(np.array([0., 0., 0.]))

array([-0., -0.,  1.])